## Setup the Simulator

In [ ]:
%load_ext autoreload
%autoreload 2


import os
import json
import numpy as np
import sys
sys.path.append('simulation')
sys.path.append('routines')
from GraphReader import GraphReader, init_graph_file, scene_num
from ProgramExecutor import execute_program

init_graph = GraphReader(graph_file=init_graph_file)
print(f'Using scene {int(scene_num)-1}, i.e. \'TestScene{scene_num}\'')

In [ ]:
print('List of all objects : ')
for room, stuff in init_graph.expanded_nodes_by_room.items():
    print('##### ',room,' #####')
    objs = list(stuff.keys())
    objs.sort()
    print(objs)

## Add your name

In [ ]:
with open('routines/ActivityObjects.json') as f:
    activities = list(json.load(f).keys())

name = input('Enter your name')

## Pick your Activity

Activity options : 

`getting_out_of_bed, brushing_teeth, showering, getting_dressed, leave_home, going_to_the_bathroom, breakfast, lunch, dinner, reading, computer_work, watching_tv, hand_wash_clothes, laundry, kitchen_cleaning, vaccuum_cleaning, wash_dishes, take_out_trash, taking_medication, sleeping, cleaning, listening_to_music, socializing, diary_logging, playing_music, coming_back_home`

In [ ]:
activity = input('Which are you scripting?')
directory = os.path.join('routines/data/sourcedScripts',name,activity)
if activity not in activities:
    raise KeyError('Are you sure you typed the correct activity? Please try again...')

activity_dir = os.path.join('routines/data/sourcedScripts',name,activity)
if not os.path.exists(activity_dir):
    os.makedirs(activity_dir)

with open('routines/ActivityObjects.json') as f:
    objs = json.load(f)
    available_obj_for_activity = objs[activity] + objs["all_activities"]

print('Objects suitable for the chosen activity are : ')
for room, stuff in init_graph.expanded_nodes_by_room.items():
    available_obj = [full_obj for full_obj,(obj,id) in list(stuff.items()) if obj in available_obj_for_activity]
    available_obj.sort()
    print('##### ',room,' #####')
    print(available_obj)

# Write the Program <br>

### Write the desired program in `program.txt`. The program should include a list of actions and comments specifying how much time you expect it to take.
Example
```
## 1-2
[Walk] <dining_room>
[Walk] <food_apple>
## 3-4
[Find] <food_apple>
[Grab] <food_apple>
[Walk] <kitchen_counter>
[PutBack] <food_apple> <kitchen_counter>
```
#### Note: Every comment must start with `##` and be of the form `<min_time_in_mins> - <max_time_in_mins>`

You will find the list of objects you can use above this cell. Here is a list of actions you can use : <br>
`WALK, FIND, SIT, STANDUP, GRAB, OPEN, CLOSE, PUTBACK, PUTIN, SWITCHON, SWITCHOFF, DRINK, LOOKAT, TURNTO, WIPE, RUN, PUTON, PUTOFF, GREET, DROP, READ, POINTAT, TOUCH, LIE, PUTOBJBACK, POUR, TYPE, WATCH, PUSH, PULL, MOVE, RINSE, WASH, SCRUB, SQUEEZE, PLUGIN, PLUGOUT, CUT, EAT, SLEEP, WAKEUP, RELEASE`

Once you are done writing the program, run the below cell to check if it works.

In [ ]:
action_headers, graphs = execute_program(program_file='program.txt', graph_file=init_graph_file, node_map=init_graph.node_map)

## If everything looks good, please show us the script and we will move the script to your database :)

In [ ]:
existing_files = [int(os.path.splitext(file)[0]) for file in os.listdir(activity_dir)]
if existing_files:
    new_file = os.path.join(activity_dir, str(max(existing_files)+1)+'.txt')
else:
    new_file = os.path.join(activity_dir, '0.txt')
print(f'Saving your awesome script at {new_file}')
os.rename('program.txt',new_file)
open('program.txt', 'a').close()

In [ ]:
! cat $new_file

### In case you need to remove your script

In [ ]:
# os.remove(new_file)

# Hurray! You did it :)